In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# spark = SparkSession.builder \
#     .appName("Kafka to Minio") \
#     .getOrCreate()

spark = SparkSession.builder \
    .appName("Kafka to Minio") \
    .config("spark.hadoop.fs.s3a.endpoint", "minio:9000") \
    .config("spark.hadoop.s3a.access.key", "ksLR8hV4x3Zm8LCe") \
    .config("spark.hadoop.fs.s3a.secret.key", "3JZFlb0fHRvQ5p6BgLn74HxqfNxrA0km") \
    .config("spark.hadoop.fs.s3a.path.style.access", True) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka-broker:9092") \
    .option("subscribe", "user") \
    .load()

df = df.selectExpr("CAST(value AS STRING)")

query = df \
    .writeStream \
    .format("json") \
    .option("path", "s3a://raw/user") \
    .option("checkpointLocation", "/tmp/checkpoints") \
    .trigger(processingTime='30 seconds') \
    .start()

query.awaitTermination()

# /tmp/checkpoints

